In [3]:
import pandas as pd
import pandas_ta as ta
import ccxt, yfinance
import requests


exchange = ccxt.binance()

In [13]:
symbols = ["ETH/USDT", "BTC/USDT", "SOL/USDT", "AVAX/USDT"]

symbols = pd.DataFrame(symbols, columns = ["symbol"])
symbols = dft2["symbol"].values.tolist()

for symbol in symbols:
    bars = exchange.fetch_ohlcv(symbol, timeframe = "5m", limit = 500)
    df = pd.DataFrame(bars, columns = ["time", "open", "high", "low", "close", "volume"])

    # technical indicators
    adx = df.ta.adx()
    macd = df.ta.macd(fast = 14, slow = 28)
    rsi = df.ta.rsi()

    # create composite df
    df = pd.concat([df, adx, macd, rsi], axis = 1)

    # grab the last value
    last_row = df.iloc[-1]

    # hook to discord channel
    WEBHOOK_URL = "https://discord.com/api/webhooks/981984523317301279/AZEt7Fz5WWZoHqUgaKfczzmj7Mjf7XuumJLLss-nEFuSZ6FWE_dUWgqvzLQpTmBAoxz1"

    # logic
    if last_row["ADX_14"] > 25:
        if last_row["DMP_14"] > last_row["DMN_14"]:
            message = f"STRONG UP TREND: {symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"
        if last_row["DMN_14"] > last_row["DMP_14"]:
            message = f"STRONG DOWN TREND:{symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"

        payload = {
            "username" : "alertbot",
            "content" : message
        }

        print(message)
        #requests.post(WEBHOOK_URL, json = payload)

    if last_row["ADX_14"] < 25:
        message = f"NO TREND:{symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"

        payload = {
            "username" : "alertbot",
            "content" : message
        }
        
        print(message)
        #requests.post(WEBHOOK_URL, json = payload)

NO TREND:ETH/USDT: The ADX is  13.17 +DI  19.18 -DI  21.73
NO TREND:BTC/USDT: The ADX is  13.27 +DI  20.87 -DI  19.95
NO TREND:SOL/USDT: The ADX is  8.41 +DI  19.22 -DI  12.90
NO TREND:AVAX/USDT: The ADX is  13.15 +DI  28.87 -DI  13.35


In [16]:
dft = pd.DataFrame(symbols, columns = ["symbol"])
dft

,symbol
0,ETH/USDT
1,BTC/USDT
2,SOL/USDT
3,AVAX/USDT


In [18]:
dft.to_csv("symbols.csv")

In [28]:
dft2 = pd.read_csv("symbols.csv")
dft2

,Unnamed: 0,symbol
0,0,ETH/USDT
1,1,BTC/USDT
2,2,SOL/USDT
3,3,AVAX/USDT


In [30]:
dft2["symbol"].values.tolist()


['ETH/USDT', 'BTC/USDT', 'SOL/USDT', 'AVAX/USDT']

In [22]:
symbols

['ETH/USDT', 'BTC/USDT', 'SOL/USDT', 'AVAX/USDT']